[묹제]
x값이 [[1,11,7,9], [1,3,4,3], [1,1,0,1]] 일때, 분류 결과를 출력하시오.

x, y = placeholder

클래스 개수 = 3

w, b = 변수 (랜덤값 초기화)

learning_rate = 0.1

트레이닝 횟수 = 2001

In [23]:
import numpy as np
import tensorflow as tf
import math

In [3]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],#2
          [0, 0, 1],#2
          [0, 0, 1],#2
          [0, 1, 0],#1
          [0, 1, 0],#1
          [0, 1, 0],#1
          [1, 0, 0],#0
          [1, 0, 0]]#0

In [12]:
tf.set_random_seed(777)
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
w = tf.Variable(tf.random_normal([4,3]))
b = tf.Variable(tf.random_normal([1]))
z = tf.matmul(x,w)+b
hf = tf.nn.softmax(z)
cost = tf.reduce_mean(tf.reduce_sum(y*-tf.log(hf), axis = 1 ))

optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(2001) :
    sess.run(train, feed_dict = {x:x_data, y:y_data})
    if i % 200 == 0 :
        print(i, sess.run(cost, feed_dict = {x:x_data, y:y_data}))

0 5.660192
200 0.649662
400 0.56876093
600 0.49894243
800 0.43122452
1000 0.36356777
1200 0.3060515
1400 0.28526682
1600 0.2682089
1800 0.2530587
2000 0.23949078


In [15]:
yhat = sess.run(hf, feed_dict = {x:[[1,11,7,9], [1,3,4,3], [1,1,0,1]]})
print("\nyhat\n",yhat)


yhat
 [[1.6835734e-03 9.9827087e-01 4.5611680e-05]
 [9.4374204e-01 5.5795759e-02 4.6220343e-04]
 [8.9847833e-07 5.8458028e-03 9.9415332e-01]]


In [18]:
yhat2 = sess.run(tf.argmax(yhat, axis = 1))
print("예측값 : ", yhat2)
y2 = sess.run(tf.argmax(y_data, axis = 1))
print("실제값 : ",y2)

예측값 :  [1 0 2]
실제값 :  [2 2 2 1 1 1 0 0]


* * *

# 동물 분류

In [25]:
xy = np.loadtxt('DataSet/zoo.csv', delimiter = ',', dtype = np.float32)
xdata = xy[:, 0:-1]
ydata = xy[:, [-1]]
print(xdata.shape, ydata.shape)


(101, 16) (101, 1)


In [37]:
nb_classes = 7
x = tf.placeholder(tf.float32, [None, 16])
y = tf.placeholder(tf.int32, [None, 1]) #y에는 0~6 사이의 임의의 수 저장(원 핫 인코딩 필요)

y_one_hot = tf.one_hot(y, nb_classes) # 0 -> 100000, 3 -> 0001000
print("one hot 상태 : ", y_one_hot)

'''
원 핫 인코딩을 수행하면 차원이 1 증가(y : (None,1) -> (None,1,7))
reshape필요 -> (None, 7)
'''

y_one_hot = tf.reshape(y_one_hot, [-1, nb_classes])
print("reshape 결과 : ", y_one_hot)

one hot 상태 :  Tensor("one_hot_5:0", shape=(?, 1, 7), dtype=float32)
reshape 결과 :  Tensor("Reshape_2:0", shape=(?, 7), dtype=float32)


In [42]:
w = tf.Variable(tf.random_normal([16,nb_classes]))
b = tf.Variable(tf.random_normal([nb_classes]))
logits = tf.matmul(x,w) + b
hf = tf.nn.softmax(logits)
cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(cost)

prediction = tf.argmax(hf, 1) # axis = 1
correct_prediction = tf.equal(prediction, tf.argmax(y_one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    for step in range(2001):
        sess.run(optimizer, feed_dict={x:xdata, y:ydata})
        if step%200==0:
            cv, av = sess.run([cost, accuracy], feed_dict={x:xdata, y:ydata})
            print(step, cv, av)
    print("예측 동물 : ", sess.run(prediction, feed_dict={x:[[0.,0.,1.,0.,0.,
                                            1.,1.,1.,1.,0.,
                                            0.,1.,0.,1.,0.,0.]]}))

0 10.077507 0.02970297
200 0.5486735 0.8910891
400 0.2960701 0.9009901
600 0.18900605 0.980198
800 0.13623165 0.990099
1000 0.106259316 1.0
1200 0.08720473 1.0
1400 0.07406866 1.0
1600 0.06447227 1.0
1800 0.05715497 1.0
2000 0.051389027 1.0
예측 동물 :  [3]
